---
## 1. Setup

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

RAW_DIR = "../data/raw"
PROCESSED_DIR = Path("../data/processed")

In [2]:
# Debug: verify RAW_DIR and list CSV files
import os

print("RAW_DIR:", os.path.abspath(RAW_DIR))
print("\nCSV files found:")
for f in sorted(Path(RAW_DIR).glob("*.csv")):
    print(f"  {f.name}")

RAW_DIR: c:\MyProjects\uidai-asris\data\raw

CSV files found:
  api_data_aadhar_biometric_1.csv
  api_data_aadhar_biometric_2.csv
  api_data_aadhar_biometric_3.csv
  api_data_aadhar_biometric_4.csv
  api_data_aadhar_demographic_1.csv
  api_data_aadhar_demographic_2.csv
  api_data_aadhar_demographic_3.csv
  api_data_aadhar_demographic_4.csv
  api_data_aadhar_demographic_5.csv
  api_data_aadhar_enrolment_1.csv
  api_data_aadhar_enrolment_2.csv
  api_data_aadhar_enrolment_3.csv


---
## 2. Connect to DuckDB

In [3]:
# Create in-memory DuckDB connection
# For persistence, use: con = duckdb.connect("../data/duckdb/uidai.duckdb")
con = duckdb.connect()

print("DuckDB connection established (in-memory)")

DuckDB connection established (in-memory)


---
## 3. Enrolment Aggregation

Read all `api_data_aadhar_enrolment_*.csv`, parse date, create `year_month`, aggregate by `[state, district, year_month]`.

In [4]:
# Enrolment: read CSVs via glob, parse date (dd-mm-yyyy), create year_month, aggregate
con.sql("""
    CREATE OR REPLACE VIEW enrol_agg_duckdb AS
    SELECT
        state,
        district,
        strftime(date, '%Y-%m') AS year_month,
        SUM(age_0_5) AS age_0_5,
        SUM(age_5_17) AS age_5_17,
        SUM(age_18_greater) AS age_18_greater
    FROM read_csv_auto('../data/raw/api_data_aadhar_enrolment_*.csv', header=true)
    GROUP BY state, district, year_month
""")

print("enrol_agg_duckdb view created")

enrol_agg_duckdb view created


In [5]:
# Inspect enrol_agg_duckdb: first 5 rows
con.sql("SELECT * FROM enrol_agg_duckdb LIMIT 5").df()

,state,district,year_month,age_0_5,age_5_17,age_18_greater
0,Odisha,Dhenkanal,2025-09,1027.0,255.0,0.0
1,Punjab,Kapurthala,2025-09,380.0,64.0,64.0
2,Rajasthan,Ganganagar,2025-09,1464.0,551.0,9.0
3,Tamil Nadu,Tenkasi,2025-09,380.0,43.0,0.0
4,Bihar,Nalanda,2025-09,912.0,2759.0,16.0


---
## 4. Demographic & Biometric Aggregation

Same pattern: read CSVs, parse date, create `year_month`, aggregate counts.

In [6]:
# Demographic: read CSVs via glob, parse date, create year_month, aggregate
con.sql("""
    CREATE OR REPLACE VIEW demo_agg_duckdb AS
    SELECT
        state,
        district,
        strftime(date, '%Y-%m') AS year_month,
        SUM(demo_age_5_17) AS demo_age_5_17,
        SUM(demo_age_17_) AS demo_age_17_
    FROM read_csv_auto('../data/raw/api_data_aadhar_demographic_*.csv', header=true)
    GROUP BY state, district, year_month
""")

print("demo_agg_duckdb view created")

demo_agg_duckdb view created


In [7]:
# Inspect demo_agg_duckdb: first 5 rows
con.sql("SELECT * FROM demo_agg_duckdb LIMIT 5").df()

,state,district,year_month,demo_age_5_17,demo_age_17_
0,Rajasthan,Udaipur,2025-03,2318.0,58055.0
1,Karnataka,Belgaum,2025-03,6671.0,12317.0
2,Tamil Nadu,Chennai,2025-03,2515.0,28888.0
3,Tamil Nadu,Salem,2025-03,2020.0,25828.0
4,Gujarat,Sabarkantha,2025-03,2777.0,10632.0


In [8]:
# Biometric: read CSVs via glob, parse date, create year_month, aggregate
con.sql("""
    CREATE OR REPLACE VIEW bio_agg_duckdb AS
    SELECT
        state,
        district,
        strftime(date, '%Y-%m') AS year_month,
        SUM(bio_age_5_17) AS bio_age_5_17,
        SUM(bio_age_17_) AS bio_age_17_
    FROM read_csv_auto('../data/raw/api_data_aadhar_biometric_*.csv', header=true)
    GROUP BY state, district, year_month
""")

print("bio_agg_duckdb view created")

bio_agg_duckdb view created


In [9]:
# Inspect bio_agg_duckdb: first 5 rows
con.sql("SELECT * FROM bio_agg_duckdb LIMIT 5").df()

,state,district,year_month,bio_age_5_17,bio_age_17_
0,Telangana,Nalgonda,2025-09,6057.0,6535.0
1,Telangana,Warangal,2025-09,5626.0,4377.0
2,Uttar Pradesh,Auraiya,2025-09,3985.0,1536.0
3,Uttar Pradesh,Bara Banki,2025-09,15545.0,4420.0
4,Uttar Pradesh,Farrukhabad,2025-09,6356.0,1762.0


---
## 5. Join All Three Aggregates

Inner join `enrol_agg_duckdb`, `demo_agg_duckdb`, `bio_agg_duckdb` on `[state, district, year_month]`.

In [10]:
# Join all three aggregates into one wide panel
con.sql("""
    CREATE OR REPLACE VIEW district_month_panel_duckdb AS
    SELECT
        e.state,
        e.district,
        e.year_month,
        e.age_0_5,
        e.age_5_17,
        e.age_18_greater,
        d.demo_age_5_17,
        d.demo_age_17_,
        b.bio_age_5_17,
        b.bio_age_17_
    FROM enrol_agg_duckdb e
    INNER JOIN demo_agg_duckdb d
        ON e.state = d.state
        AND e.district = d.district
        AND e.year_month = d.year_month
    INNER JOIN bio_agg_duckdb b
        ON e.state = b.state
        AND e.district = b.district
        AND e.year_month = b.year_month
""")

print("district_month_panel_duckdb view created")

district_month_panel_duckdb view created


In [11]:
# Check row count and preview
row_count = con.sql("SELECT COUNT(*) AS cnt FROM district_month_panel_duckdb").df().iloc[0, 0]
print(f"district_month_panel_duckdb row count: {row_count}")

con.sql("SELECT * FROM district_month_panel_duckdb LIMIT 5").df()

district_month_panel_duckdb row count: 4355


,state,district,year_month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,Karnataka,yadgir,2025-12,77.0,16.0,0.0,92.0,674.0,146.0,664.0
1,Kerala,Kozhikode,2025-12,985.0,305.0,27.0,2173.0,23737.0,9513.0,11858.0
2,Kerala,Palakkad,2025-12,1318.0,212.0,24.0,880.0,14708.0,5875.0,10697.0
3,Madhya Pradesh,Alirajpur,2025-12,561.0,310.0,113.0,561.0,4014.0,2564.0,3274.0
4,Madhya Pradesh,Guna,2025-12,1379.0,292.0,19.0,741.0,6482.0,4505.0,5577.0


---
## 6. Export to Pandas & Save CSV

In [12]:
# Export DuckDB result to Pandas DataFrame
district_month_panel = con.sql("SELECT * FROM district_month_panel_duckdb").df()

print(f"district_month_panel.shape: {district_month_panel.shape}")
print(f"Columns: {district_month_panel.columns.tolist()}")
district_month_panel.head()

district_month_panel.shape: (4355, 10)
Columns: ['state', 'district', 'year_month', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_']


,state,district,year_month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,Kerala,Ernakulam,2025-12,679.0,243.0,87.0,1282.0,16957.0,8645.0,8794.0
1,Madhya Pradesh,Chhindwara,2025-12,1813.0,100.0,2.0,1446.0,8951.0,11688.0,9179.0
2,Madhya Pradesh,Satna,2025-12,1845.0,281.0,58.0,1640.0,10514.0,11263.0,9590.0
3,Madhya Pradesh,Tikamgarh,2025-12,1507.0,263.0,8.0,1324.0,7964.0,6769.0,6616.0
4,Maharashtra,Beed,2025-12,881.0,93.0,18.0,1924.0,21465.0,16845.0,12408.0


In [13]:
# Ensure processed directory exists and save CSV
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

output_path = PROCESSED_DIR / "district_month_panel_duckdb.csv"
district_month_panel.to_csv(output_path, index=False)

print(f"Saved to: {output_path.resolve()}")

Saved to: C:\MyProjects\uidai-asris\data\processed\district_month_panel_duckdb.csv


In [14]:
# Close DuckDB connection
con.close()
print("DuckDB connection closed")

DuckDB connection closed


---
## Phase 2 (DuckDB) Summary

**What was done in this notebook:**

- **DuckDB read all raw CSVs using glob patterns:**
  - `api_data_aadhar_enrolment_*.csv` → `enrol_agg_duckdb`
  - `api_data_aadhar_demographic_*.csv` → `demo_agg_duckdb`
  - `api_data_aadhar_biometric_*.csv` → `bio_agg_duckdb`

- **SQL created `year_month` and aggregated at district × month:**
  - Parsed `date` from `dd-mm-yyyy` using `strptime`
  - Created `year_month` as `"YYYY-MM"` using `strftime`
  - Grouped by `[state, district, year_month]` with SUM of count columns

- **Aggregated panel joined and exported:**
  - Inner join of all three views → `district_month_panel_duckdb`
  - Exported to Pandas DataFrame → `district_month_panel`
  - Saved as CSV: `../data/processed/district_month_panel_duckdb.csv`

---

*Ready for Phase 3: EDA, feature engineering, or modeling.*